# Application of VectorDB and Embeddings

# TL;DR: Token Embeddings vs Vector Databases

## Key Distinction

**Token/Word Embeddings** = Tool used to create vectors  
**Vector Database** = Storage for final document vectors

---

## Where Are Token Embeddings Stored?

**Inside the model weight files** (e.g., `pytorch_model.bin`, `model.safetensors`)
```python
# Token embeddings are model parameters
model.embeddings.word_embeddings.weight
# Shape: [vocab_size, embedding_dim]
# Example: [50,000 tokens × 768 dimensions]
```

**Physical location:** Model checkpoint files (~440 MB for BERT)

---

## Vector DB vs Model Training

### Training a Transformer (No Vector DB)
- Train from scratch on raw text
- Embeddings learned and stored in **model weights**
- Output: Trained model file
- **NO VECTOR DB INVOLVED**

### RAG Application (Uses Vector DB)
- Use **pre-trained** model
- Generate embeddings for documents
- Store document vectors in **Vector DB** (Pinecone, Weaviate, Chroma)
- Retrieve relevant docs at query time

---

## What Goes in Vector DB?

**NOT stored:** Individual token embeddings  
**Stored:** Document/chunk embeddings
```
Input: "Machine learning is powerful"
↓
Model processes using token embeddings internally
↓
Output: Single vector [0.42, 0.28, 0.35, ...] ← Stored in Vector DB
```

---

## Simple Analogy

- **Token embeddings** = Knife (tool)
- **Document embeddings** = Chopped vegetables (product stored in container)
- **Vector DB** = Container/refrigerator (storage system)
- **Model training** = Learning to use the knife (no storage needed)
- **RAG** = Using the knife to prep food, then storing it (storage needed)

---

## Bottom Line

1. Token embeddings live **inside model files** as learned parameters
2. Vector DBs store **document embeddings** created by those models
3. Training = no Vector DB; RAG = uses Vector DB

# What is Embedding

## Summary: Embeddings in NLP (Traditional → Neural → Transformer)

### 1. What is an Embedding?
An **embedding** is a numerical (dense vector) representation of a discrete object (e.g., a word) that captures useful information for machine learning models.

- Purpose: Convert symbols → numbers
- Meaning is encoded in **geometry (distances & directions)**, not individual values

---

### 2. What is a Word Embedding?
A **word embedding** maps each word to a vector such that:
- Semantically similar words are close in vector space
- Dissimilar words are far apart

Examples:
- Word2Vec
- GloVe
- FastText

---

### 3. Embedding Dimension: Minimum and Maximum

#### Minimum
- **Theoretical minimum:** 1 (not useful)
- **Practical minimum:** ~10–50 (toy or simple tasks)

#### Maximum
- **Theoretical maximum:** Unlimited
- **Practical range:**
  - Static embeddings: 100–300
  - Transformers: 768–12,000+

Trade-off:
- Too small → insufficient capacity
- Too large → overfitting, inefficiency

---

### 4. Meaning of Each Number in an Embedding

Key truth:
> **Individual embedding dimensions have no fixed human-interpretable meaning.**

- Embeddings are **distributed representations**
- Meaning emerges from:
  - Distances
  - Angles
  - Directions between vectors
- Dimensions are **rotation-invariant** and arbitrary

Meaning lives in **relationships**, not coordinates.

---

### 5. How Are Embeddings Trained?

#### Core principle
**Distributional hypothesis**:
> Words appearing in similar contexts have similar meanings.

#### Neural embeddings (most common)
- Trained via **gradient descent**
- Objective: Predict context or next word
- Examples:
  - Word2Vec (Skip-gram, CBOW)
  - FastText
  - BERT, GPT

#### Non-gradient embeddings
- LSA (SVD-based)
- Spectral embeddings
- TF-IDF (not really embeddings)

---

### 6. Traditional NLP vs Transformer NLP

#### Traditional NLP
- Representations:
  - Bag-of-Words
  - TF-IDF
  - N-grams
- Position:
  - N-grams
  - Sliding windows
- Interaction:
  - Hand-crafted features
  - RNN/LSTM recurrence
- Pipelines were **modular and non end-to-end**

#### Transformer NLP
- Token embeddings
- Positional embeddings
- Attention mechanism
- End-to-end learned representations
- Contextual embeddings

**Key difference:**
> Traditional NLP separates meaning, position, and interaction; transformers learn them jointly.

---

### 7. Are All Embeddings Trained with Gradient Descent?

**No.**

| Embedding Type | Gradient Descent |
|---|---|
| Word2Vec | Yes |
| FastText | Yes |
| GloVe | Yes |
| BERT / GPT | Yes |
| LSA | No |
| TF-IDF | No |
| Spectral embeddings | No |
| Random embeddings | No |

Modern NLP is dominated by gradient-based methods due to scalability and flexibility.

---

### 8. Embeddings in RNN / LSTM Models

Key point:
> **Embeddings are crucial in RNN/LSTM-based NLP models.**

- Embeddings provide semantic signal
- RNN/LSTM:
  - Models order
  - Aggregates information over time
- Cannot compensate for poor embeddings (GIGO principle)

Empirical finding:
- Pre-trained embeddings often contribute more than the RNN itself

---

### 9. Pre-trained Embeddings: Frozen vs Fine-Tuned

Pre-trained embeddings are **initializations**, not fixed by default.

#### Option 1: Frozen
- No updates during training
- Good for small datasets
- Faster, more stable

#### Option 2: Fine-tuned
- Updated via backpropagation
- Adapts to task/domain
- Risk of overfitting or forgetting

Best practice:
- Small data → freeze
- Larger or domain-specific data → fine-tune (often with smaller LR)

---

### 10. Big Picture Takeaways

- Embeddings turn language into geometry
- Individual dimensions are meaningless; geometry is everything
- Gradient descent dominates modern embedding learning
- In RNN/LSTM models, embeddings carry most semantic power
- Transformers reduce reliance on static embeddings via attention
- Pre-trained embeddings can (and often should) be fine-tuned

---

### One-Sentence Summary

**Embeddings are learned geometric representations of language; how powerful your NLP model is largely depends on how well those vectors encode meaning and context.**




# Skip-gram with a Simple Numerical Example

I’ll tell it like it is:

Real skip-gram training uses gradient descent and softmax over large vocabularies. Doing full training by hand is ugly.  
So we’ll use a **tiny toy corpus**, **2D embeddings**, and show **one concrete skip-gram update intuition with numbers**.

---

## 1️⃣ Tiny corpus (toy data)

**Sentence:**

> “I like cats”

**Vocabulary (index → word):**





**Window size = 1**

### Skip-gram pairs (center → context)

| Center | Context |
|------|--------|
| like | I |
| like | cats |

So the model learns:

> If the center word is **“like”**, it should predict **“I”** and **“cats”**

---

## 2️⃣ What skip-gram actually learns (no magic)

Skip-gram learns **two embeddings per word**:

- **Input vector** (center word)
- **Output vector** (context word)

We’ll use **2D embeddings** to visualize easily.

---

## 3️⃣ Initialize embeddings (random, small numbers)

### Input vectors (V)

| Word | Vector |
|----|------|
| I | `[0.2, 0.1]` |
| like | `[0.0, 0.3]` |
| cats | `[0.4, 0.2]` |

### Output vectors (U)

| Word | Vector |
|----|------|
| I | `[0.1, 0.0]` |
| like | `[0.0, 0.2]` |
| cats | `[0.3, 0.1]` |

---

## 4️⃣ Skip-gram prediction (core calculation)

We take **center word = "like"**

### Step 1: Dot product with each context word

**Formula:**

\[
\text{score}(w_c, w_o) = \mathbf{v}_{w_c} \cdot \mathbf{u}_{w_o}
\]

### Dot products

**score(like, I):**

\[
[0.0, 0.3] \cdot [0.1, 0.0] = 0.0
\]

**score(like, cats):**

\[
[0.0, 0.3] \cdot [0.3, 0.1] = 0.03
\]

**score(like, like):**

\[
[0.0, 0.3] \cdot [0.0, 0.2] = 0.06
\]

---

## 5️⃣ Convert scores to probabilities (softmax)

\[
P(w_o \mid w_c) =
\frac{e^{\text{score}}}{\sum e^{\text{scores}}}
\]

### Exponentials

| Word | exp(score) |
|----|----|
| I | \(e^0 = 1\) |
| cats | \(e^{0.03} \approx 1.03\) |
| like | \(e^{0.06} \approx 1.06\) |

**Sum ≈ 3.09**

### Probabilities

| Word | Probability |
|----|----|
| I | \(1 / 3.09 \approx 0.32\) |
| cats | \(1.03 / 3.09 \approx 0.33\) |
| like | \(1.06 / 3.09 \approx 0.35\) |

❌ **Problem:**  
The model predicts **“like”** as context, which is wrong.

---

## 6️⃣ What learning does (plain English)

Skip-gram will:

- Pull **“like”** closer to **“I”** and **“cats”**
- Push **“like”** away from unrelated words

After many updates, embeddings might look like this:

---

## 7️⃣ Final learned 2D embeddings (intuitive result)

### Input embeddings (after training)

| Word | Vector |
|----|------|
| I | `[-0.2, 0.3]` |
| like | `[0.0, 0.5]` |
| cats | `[0.2, 0.4]` |

### Visual intuition (2D space)




✔ Words appearing together are **geometrically close**

---

## 8️⃣ Simple analogy (remember this)

- Skip-gram is like **learning a map of words**
- Words that appear together are **neighbors on the map**
- Training = repeatedly nudging words **closer or farther apart**

No linguistics.  
No grammar rules.  
Just **geometry + statistics**.

---

## 9️⃣ Key takeaway (don’t sugar-coat it)

- Word embeddings are **not semantic by design**
- Meaning **emerges from co-occurrence**
- Skip-gram is just:

> “Adjust vectors so dot products predict nearby words”

---

## 📚 References (foundational, authoritative)

- Mikolov et al., *Efficient Estimation of Word Representations in Vector Space*, 2013  
- Goldberg & Levy, *word2vec Explained*, 2014  
- Jurafsky & Martin, *Speech and Language Processing*, Chapter on Vector Semantics  

---

If you want next, I can:

- Show **negative sampling numerically**
- Compare **CBOW vs Skip-gram**
- Plot this in **Python**
- Explain why embeddings capture **analogies (king − man + woman)**


# Document Embedding: Complete Guide

## Table of Contents
1. [What is Document Embedding](#what-is-document-embedding)
2. [How to Use Pre-trained Models](#how-to-use-pre-trained-models)
3. [Applications Beyond Search](#applications-beyond-search)
4. [Vector Database Use Cases](#vector-database-use-cases)
5. [Critical Considerations](#critical-considerations)

---

## What is Document Embedding

### Definition
Document embedding converts entire documents (articles, emails, paragraphs) into a single vector of numbers that captures its meaning.

**Analogy**: Like a fingerprint for text - documents with similar meanings get similar fingerprints.

### Basic Process
```
"The cat sat on the mat" → [0.23, -0.45, 0.67, 0.12, ..., -0.31]
```

Typical dimensions: 300-1536 (or more)

### Why It Matters
- **Find similar documents** - semantic grouping
- **Search by meaning** - not just keywords
- **Cluster documents** - automatic categorization
- **Feed into ML models** - convert text to numbers

### Simple Example
```
Document 1: "dogs are pets" → [0.8, 0.1, 0.2]
Document 2: "cats are pets" → [0.7, 0.2, 0.3]
Document 3: "cars need fuel" → [0.1, 0.9, 0.5]

similarity(Doc1, Doc2) = 0.97  # very similar
similarity(Doc1, Doc3) = 0.52  # less similar
```

---

## How to Use Pre-trained Models

### Key Concept
Pre-trained models don't need prior embedding information - they create embeddings from raw text.

**Analogy**: Like a meat grinder - feed raw text in, get embeddings out. The model was already "trained" to process text properly.

### Basic Workflow

```python
from transformers import AutoTokenizer, AutoModel

# 1. Load pre-trained model
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# 2. Your raw documents
documents = [
    "Machine learning is a subset of AI",
    "Deep learning uses neural networks"
]

# 3. Generate embeddings
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", 
                      padding=True, truncation=True)
    outputs = model(**inputs)
    embedding = outputs.last_hidden_state.mean(dim=1)
    return embedding.squeeze().numpy()

embeddings = [get_embedding(doc) for doc in documents]
```

### Internal Process
1. **Tokenization**: Text → Token IDs
2. **Model processing**: Token IDs → Contextual representations
3. **Pooling**: Multiple token vectors → Single document vector

---

## Applications Beyond Search

### 1. Document Classification
Train classifiers on embeddings to categorize documents.

```python
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(train_embeddings, labels)
prediction = classifier.predict([new_embedding])
```

**Use cases**: Email filtering, content moderation, support ticket routing

### 2. Clustering & Topic Discovery
Group similar documents without predefined categories.

```python
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=10)
clusters = kmeans.fit_predict(embeddings)
```

**Use cases**: Organizing document collections, discovering themes in feedback

### 3. Anomaly Detection
Find documents that don't fit normal patterns.

```python
from sklearn.ensemble import IsolationForest

detector = IsolationForest()
detector.fit(normal_embeddings)
is_anomaly = detector.predict([new_embedding])
```

**Use cases**: Fraud detection, spotting unusual reports, content moderation

### 4. Recommendation Systems
Recommend documents similar to user's interests.

```python
user_profile = np.mean([get_embedding(doc) for doc in user_history], axis=0)
similarities = cosine_similarity([user_profile], all_embeddings)
top_recommendations = np.argsort(similarities[0])[-5:]
```

**Use cases**: News recommendations, research paper suggestions

### 5. Information Extraction & QA
Find which document/section contains the answer.

```python
q_embedding = get_embedding(question)
similarities = cosine_similarity([q_embedding], section_embeddings)
best_section = sections[np.argmax(similarities)]
```

**Use cases**: Customer support bots, legal document analysis

### 6. Duplicate Detection
Find near-duplicate or plagiarized content.

```python
similarity = cosine_similarity([doc1_emb], [doc2_emb])[0][0]
if similarity > 0.95:
    print("Likely duplicate")
```

**Use cases**: Content moderation, plagiarism checking, deduplication

### 7. Transfer Learning
Use embeddings as features for other ML tasks.

```python
model = RandomForestRegressor()
model.fit(embeddings, engagement_scores)
predicted_engagement = model.predict([new_emb])
```

**Use cases**: Predicting virality, reading time estimation

### 8. Multi-modal Applications
Combine with image/audio embeddings.

**Use cases**: Image captioning, video search, cross-lingual search

### 9. Data Augmentation
Generate synthetic examples by interpolating embeddings.

```python
synthetic_emb = 0.5 * emb1 + 0.5 * emb2
```

**Use cases**: Expanding training datasets, data balancing

---

## Vector Database Use Cases

### Beyond RAG Applications

#### 1. Recommendation Systems
Store user and item embeddings for real-time recommendations.

```python
# Find similar products
results = collection.query(
    query_embeddings=[user_interest],
    n_results=5
)
```

**Real use**: Netflix shows, Spotify songs, Amazon products, LinkedIn jobs

**Reference**: Pinterest uses vector search for visual recommendations (Pinterest Engineering Blog, 2019)

#### 2. Duplicate/Near-Duplicate Detection
Find similar content at scale.

```python
duplicates = collection.query(
    query_embeddings=[new_embedding],
    where={"status": "open"}
)
```

**Real use**: Content moderation, legal document review, code plagiarism

**Reference**: Airbnb uses embeddings for duplicate listing detection (KDD 2018)

#### 3. Anomaly/Fraud Detection
Identify outliers in high-dimensional space.

```python
avg_distance = np.mean(neighbors["distances"])
if avg_distance > threshold:
    flag_as_fraud()
```

**Real use**: Credit card fraud, insurance claims, network intrusion, quality control

**Reference**: PayPal uses embeddings for fraud detection (IEEE Symposium, 2016)

#### 4. Personalization & User Modeling
Store and update user preference vectors over time.

```python
# Update with exponential moving average
new_emb = (1 - weight) * current_emb + weight * action_embedding
```

**Real use**: News feed personalization, adaptive learning, dating apps

#### 5. Image/Multi-modal Search
Search images by visual similarity or text description.

```python
# Search images using text query
text_embedding = model.encode_text("red sports car")
results = image_collection.query(query_embeddings=[text_embedding])
```

**Real use**: Google Images, Pinterest visual search, e-commerce product search

**Reference**: Google uses embeddings for image search (CVPR 2020)

#### 6. Version Control & Change Detection
Track how documents/code evolves.

```python
# Compare consecutive versions
if similarity < 0.7:
    print("Major change detected")
```

**Real use**: Wiki changes, code review prioritization, contract tracking

#### 7. A/B Testing & Content Optimization
Analyze what content performs well.

```python
# Predict performance based on similar content
predicted_performance = np.mean([m["open_rate"] for m in similar])
```

**Real use**: Email marketing, headline testing, ad copy optimization

#### 8. Clustering & Segmentation
Dynamically group similar items.

```python
similar_customers = collection.query(
    query_embeddings=[seed_customer.embedding],
    where={"churn_risk": "high"}
)
```

**Real use**: Customer segmentation, market research, user cohorts

#### 9. Time-Series Pattern Matching
Find similar patterns in sequential data.

```python
pattern_emb = embed_timeseries(price_pattern)
similar_patterns = collection.query(query_embeddings=[pattern_emb])
```

**Real use**: Trading patterns, predictive maintenance, weather forecasting

**Reference**: Uber uses embeddings for time-series forecasting (Uber Engineering Blog, 2020)

#### 10. Cold Start Problem Solutions
Match new users/items based on attributes.

```python
new_user_emb = get_embedding(str(user_attributes))
similar_users = collection.query(query_embeddings=[new_user_emb])
```

**Real use**: New user onboarding, new product recommendations

### Vector DB Advantages
1. **Speed**: Sub-second search on millions/billions of vectors
2. **Scale**: Handle datasets too large for memory
3. **Updates**: Real-time insertion/deletion
4. **Filtering**: Combine similarity with metadata filters
5. **Hybrid Search**: Mix keyword and semantic search

### Popular Vector Databases
- **Pinecone**: Managed, easy to use
- **Weaviate**: Open-source, ML-native
- **Milvus**: Scalable, open-source
- **Qdrant**: Rust-based, fast
- **ChromaDB**: Simple, embedded

**References**:
- Malkov & Yashunin (2018). "Efficient and robust approximate nearest neighbor search using HNSW"
- Johnson et al. (2019). "Billion-scale similarity search with GPUs" - FAISS

---

## Critical Considerations

### 1. Different Embedding Methods & Trade-offs

#### a) Averaging Word Embeddings (Old School)
```python
word_embeddings = [w2v_model[word] for word in doc.split()]
doc_embedding = np.mean(word_embeddings, axis=0)
```
- **Pro**: Fast, lightweight
- **Con**: Loses word order
- **When**: Simple tasks, limited compute

#### b) Transformer-based (Modern Standard)
```python
doc_embedding = model.encode(doc)
```
- **Pro**: Captures context, word order, semantics
- **Con**: Slower, larger models
- **When**: Most production use cases

#### c) Specialized Models
- **Dense Passage Retrieval (DPR)**: Optimized for Q&A
- **SimCSE**: Focuses on sentence similarity
- **Instructor**: Task-specific embeddings

**Key Point**: Embeddings trained for different tasks aren't interchangeable.

**Reference**: Reimers & Gurevych (2019). "Sentence-BERT" (ACL)

---

### 2. Dimensionality Matters

#### Common Sizes
- Word2Vec: 300 dimensions
- BERT-base: 768 dimensions
- BERT-large: 1024 dimensions
- OpenAI ada-002: 1536 dimensions

#### Storage Impact
```
384 dims: 1M docs × 384 × 4 bytes = ~1.5 GB
1536 dims: 1M docs × 1536 × 4 bytes = ~6 GB
```

#### Dimensionality Reduction
```python
from sklearn.decomposition import PCA

pca = PCA(n_components=256)
reduced_embeddings = pca.fit_transform(original_embeddings)
# Lose 5-15% accuracy but gain 3x speed
```

---

### 3. Context Length Limits

#### Common Limits
- BERT: 512 tokens (~400 words)
- Longformer: 4096 tokens (~3000 words)
- GPT-3 embeddings: 8191 tokens
- Latest models: up to 32k+ tokens

#### Handling Long Documents

**Option 1: Truncation**
```python
embedding = model.encode(doc[:512])  # Loses information
```

**Option 2: Chunking + Averaging**
```python
chunks = [doc[i:i+512] for i in range(0, len(doc), 512)]
chunk_embeddings = [model.encode(chunk) for chunk in chunks]
doc_embedding = np.mean(chunk_embeddings, axis=0)
```

**Option 3: Max Pooling**
```python
doc_embedding = np.max(chunk_embeddings, axis=0)
```

**Option 4: Hierarchical Embedding**
```python
para_embeddings = [model.encode(para) for para in paragraphs]
doc_embedding = meta_model.encode(para_embeddings)
```

---

### 4. Normalization is Critical

Most similarity metrics assume normalized vectors.

```python
from sklearn.preprocessing import normalize

# Correct approach
embedding = normalize(model.encode(doc).reshape(1, -1))[0]

# With normalized vectors: cosine similarity = dot product
similarity = np.dot(emb1, emb2)  # 10x faster
```

**Note**: Most modern models output normalized vectors by default, but always verify.

---

### 5. Domain Adaptation & Fine-tuning

Pre-trained models may underperform on domain-specific text.

#### When to Fine-tune
- Domain-specific jargon (legal, medical, technical)
- Unique document types (resumes, recipes, code)
- Non-English languages
- Performance isn't good enough

#### Fine-tuning Example
```python
from sentence_transformers import SentenceTransformer, InputExample, losses

model = SentenceTransformer('all-MiniLM-L6-v2')

train_examples = [
    InputExample(texts=['MI', 'myocardial infarction'], label=1.0),
    InputExample(texts=['MI', 'machine intelligence'], label=0.0),
]

model.fit(train_objectives=[(dataloader, loss)], epochs=1)
```

**Reference**: Guu et al. (2020). "REALM: Retrieval-Augmented Language Model Pre-Training" (ICML)

---

### 6. Multilingual Considerations

Not all models handle multiple languages well.

```python
multi_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

en_emb = multi_model.encode("Machine learning is powerful")
es_emb = multi_model.encode("El aprendizaje automático es poderoso")

similarity = cosine_similarity([en_emb], [es_emb])  # ~0.85
```

**Cross-lingual search**: Query in English, find documents in any language.

---

### 7. Evaluation Metrics

#### Intrinsic Evaluation (Embedding Quality)

**Semantic Textual Similarity (STS)**
```python
from scipy.stats import spearmanr

# Compare model predictions with human judgments
correlation, p_value = spearmanr(predicted_sims, human_scores)
# Good models: 0.75-0.85
```

#### Extrinsic Evaluation (Task Performance)

**Retrieval Metrics**
```python
def recall_at_k(query_emb, doc_embeddings, relevant_docs, k=10):
    similarities = cosine_similarity([query_emb], doc_embeddings)[0]
    top_k = np.argsort(similarities)[-k:]
    retrieved = set(top_k) & set(relevant_docs)
    return len(retrieved) / len(relevant_docs)
```

Common metrics: Recall@K, MRR (Mean Reciprocal Rank), NDCG

**Reference**: Muennighoff et al. (2022). "MTEB: Massive Text Embedding Benchmark"

---

### 8. Cold Start & Embedding Drift

#### Cold Start Problem
New documents have no similar neighbors initially.

**Solution**: Metadata filtering + hybrid search
```python
results = collection.query(
    query_embeddings=[new_doc_emb],
    where={"category": "tech", "date": {"$gte": "2024-01-01"}}
)
```

#### Embedding Drift
Embeddings become outdated as language/domain evolves.

**Monitoring**
```python
def check_drift(old_embeddings, new_embeddings, threshold=0.1):
    avg_old = np.mean(old_embeddings, axis=0)
    avg_new = np.mean(new_embeddings, axis=0)
    drift = 1 - cosine_similarity([avg_old], [avg_new])[0][0]
    
    if drift > threshold:
        print("Significant drift - consider retraining")
    return drift
```

**Solution**: Periodic re-embedding or continuous fine-tuning.

---

### 9. Computational Costs

Embedding isn't free - plan accordingly.

#### Optimization Strategies

**Batch Processing**
```python
# Good: batches (10x faster)
embeddings = model.encode(docs, batch_size=32)

# Bad: one at a time
for doc in docs:
    emb = model.encode(doc)
```

**GPU Acceleration**
```python
model = SentenceTransformer('all-MiniLM-L6-v2', device='cuda')
# 50-100x faster than CPU
```

**Model Distillation**
- Teacher: BERT-large (1024 dims)
- Student: MiniLM (384 dims)
- Result: 3x faster, 90% accuracy

**Caching**
```python
from functools import lru_cache

@lru_cache(maxsize=10000)
def get_embedding_cached(text):
    return model.encode(text)
```

---

### 10. Security & Privacy

Embeddings can leak information.

#### Risk Example
```python
doc = "Patient John Smith has HIV"
embedding = model.encode(doc)
# Attacker with model can approximately reconstruct sensitive info
```

#### Mitigations
- Differential privacy during training
- Encrypted embeddings (homomorphic encryption)
- Access control on vector databases
- Anonymize before embedding

**Reference**: Song & Shmatikov (2019). "Auditing Data Provenance in Text-Generation Models" (KDD)

---

### 11. When NOT to Use Embeddings

Embeddings aren't always the answer.

**Don't use when**:
- Exact keyword matching needed (legal/compliance)
- Highly structured data (use SQL)
- Tiny datasets (<1000 docs) - simpler methods work
- Real-time streaming (latency too high)
- Explainability is critical (embeddings are black boxes)

---

## Critical Pre-Deployment Checklist

✅ **Model selection**: Right model for your domain/language?  
✅ **Dimensionality**: Balanced performance vs. storage?  
✅ **Normalization**: Vectors normalized for fast similarity?  
✅ **Context length**: Documents fit in model limits?  
✅ **Evaluation**: Measured on YOUR task, not generic benchmarks?  
✅ **Scalability**: Can handle your data growth?  
✅ **Costs**: Compute/storage budgeted?  
✅ **Monitoring**: Drift detection in place?  
✅ **Privacy**: Sensitive data protected?  

---

## Key Takeaways

1. **Document embeddings convert text to numerical vectors** that capture semantic meaning
2. **Pre-trained models work on raw text** - no prior processing needed
3. **Applications go far beyond search** - classification, clustering, anomaly detection, recommendations
4. **Vector databases enable scalable similarity search** at production scale
5. **Choose the right method** - consider domain, language, task, and constraints
6. **Dimensionality matters** - balance accuracy, speed, and storage
7. **Handle long documents carefully** - chunking strategies matter
8. **Fine-tune for domain-specific tasks** when pre-trained models underperform
9. **Monitor and maintain** - embeddings can drift over time
10. **Consider costs and privacy** - both computational and security implications

---

## Further Reading

- Reimers & Gurevych (2019). "Sentence-BERT: Sentence Embeddings using Siamese BERT-Networks" (ACL)
- Guu et al. (2020). "REALM: Retrieval-Augmented Language Model Pre-Training" (ICML)
- Muennighoff et al. (2022). "MTEB: Massive Text Embedding Benchmark" (arXiv)
- Malkov & Yashunin (2018). "Efficient and robust approximate nearest neighbor search using HNSW"
- Johnson et al. (2019). "Billion-scale similarity search with GPUs" - FAISS

---

*Document created: February 2026*